# 打印html

In [1]:
import json
lf = open('answer.txt','r')
wf = open('answer.html','w')
wf.write("<html><head><meta http-equiv=\"Content-Type\" content=\"text/html;charset=utf-8\"></head><body>")
for line in lf:
    js = json.loads(line)
    wf.write("<div style='border:1px solid black;'>")
    wf.write('id=%s<br />'%js['value']['id'])
    wf.write('grade=%d<br />'%js['value']['grade'])
    wf.write('subject=%d<br />'%js['value']['subject'])
    wf.write('question=%s<br />'%js['value']['question'].encode('utf-8'))
    wf.write('knowledge=')
    for i in range(len(js['value']['knowledge'])):
        wf.write('%d '%js['value']['knowledge'][i])
    wf.write('<br />')
    if js['value']['type'] == 0:
        wf.write('options=')
        for option in js['value']['options']:
            wf.write(option.encode('utf-8')+' ')
        wf.write('<br />')
    wf.write('analysis=%s<br />'%js['value']['analysis'].encode('utf-8'))
    wf.write('answers=')
    for item in js['value']['answers']:
        wf.write(item.encode('utf-8'))
    wf.write('<br />type=%d<br />'%js['value']['type'])
    wf.write('</div><br />')
wf.write('</body></html>')
lf.close
wf.close

<function close>

# 创建数据库类

In [1]:
#encoding=utf-8
import pymssql
class MSSQL:
    def __init__(self, host, user, pwd, db):
        self.host = host
        self.user = user
        self.pwd = pwd
        self.db = db
        
    def __GetConnect(self):
        if not self.db:
            raise(NameError, '没有设置数据库信息')
        self.conn = pymssql.connect(host=self.host,user=self.user,password=self.pwd,database=self.db,charset='utf8')
        cur=self.conn.cursor()
        if not cur:
            raise(NameError,'连接数据库失败')
        else:
            return cur
        
    def ExecQuery(self, sql):
        cur = self.__GetConnect()
        cur.execute(sql)
        resList = cur.fetchall()
        self.conn.close()
        return resList
    
    def ExecNonQuery(self, sql):
        cur = self.__GetConnect()
        cur.execute(sql)
        self.conn.commit()
        self.conn.close()


# 导入数据库

In [2]:
#encoding=utf-8
import json
import re
count = 0
ms = MSSQL(host=r'(local)',user=r'DRAKE',pwd=r'123456',db=r'question')
lf = open('answer.txt','r')
for line in lf:
    js = json.loads(line)
    value = js['value']
    
    # id ntext
    ID = value['id'].encode('utf-8')
    
    # grade int
    if value.has_key('grade'):
        grade = value['grade']
    else:
        grade = None
    
    # subject int
    if value.has_key('subject'):
        subject = value['subject']
    else:
        subject = None
        
    # question ntext
    question = value['question'].encode('utf-8').replace('\'','\'\'')
    #pattern = re.compile(r'<frm>.+</frm>')
    #if pattern.match(question):
    #    print(ID)
    #    strList = re.findall(pattern, question)
    #    for item in strList:
    #        print(item)
    question = question.replace(r'<frm>', r'\(').replace(r'</frm>', r'\)')
    
    # knowledge ntext
    if value.has_key('knowledge'):
        knowledge = json.dumps(value['knowledge']).replace('\'','\'\'')
    else:
        knowledge = ''
        
    # options ntext
    optionA = ''
    optionB = ''
    optionC = ''
    optionD = ''
    optionE = ''
    if value.has_key('options'):
        length = len(value['options'])
        optionA = value['options'][0].encode('utf-8').replace('\'','\'\'').replace(r'<frm>', r'\(').replace(r'</frm>', r'\)')
        if length >= 2:
            optionB = value['options'][1].encode('utf-8').replace('\'','\'\'').replace(r'<frm>', r'\(').replace(r'</frm>', r'\)')
        if length >= 3:
            optionC = value['options'][2].encode('utf-8').replace('\'','\'\'').replace(r'<frm>', r'\(').replace(r'</frm>', r'\)')
        if length >= 4:
            optionD = value['options'][3].encode('utf-8').replace('\'','\'\'').replace(r'<frm>', r'\(').replace(r'</frm>', r'\)')
        if length >= 5:
            optionE = value['options'][4].encode('utf-8').replace('\'','\'\'').replace(r'<frm>', r'\(').replace(r'</frm>', r'\)')
    
        
    # analysis ntext
    if value.has_key('analysis'):
        analysis = value['analysis'].encode('utf-8').replace('\'','\'\'').replace(r'<frm>', r'\(').replace(r'</frm>', r'\)')
    else:
        analysis = ''
        
    # answers ntext
    answers = ''
    for answer in value['answers']:
        answers = answers + answer.encode('utf-8').replace('\'','\'\'').replace(r'<frm>', r'\(').replace(r'</frm>', r'\)')
        
    # type int
    Type = value['type']
    
    #print 'id=',ID,type(ID)
    #print 'grade=',grade,type(grade)
    #print 'subject=',subject,type(subject)
    #print 'question=',question,type(question)
    #print 'knowledge=',knowledge,type(knowledge)
    #print 'optionA=',optionA,type(optionA)
    #print 'analysis=',analysis,type(analysis)
    #print 'answers=',answers,type(answers)
    #print 'type=',Type,type(Type)
    #print ''
    
    resList = ms.ExecQuery("select * from dbo.OnlineEducationSite_question where id='%s'" % ID)
    if len(resList) == 0:
        ms.ExecNonQuery("insert into dbo.OnlineEducationSite_question values('%s',%d,  %d,'%s','%s','%s','%s','%s','%s','%s','%s','%s',%d)"%(ID,grade,subject,question,knowledge,optionA,optionB,optionC,optionD,optionE,analysis,answers,Type))
    
    count += 1
    if count % 1000 == 0:
        print '已保存%d条数据！'% count
print '保存完成！一共保存了%d条数据！'% count
lf.close




已保存1000条数据！
已保存2000条数据！
已保存3000条数据！
已保存4000条数据！
保存完成！一共保存了4976条数据！


<function close>

# 清空数据库

In [24]:
ms = MSSQL(host=r'(local)',user=r'DRAKE',pwd=r'123456',db=r'question')
ms.ExecNonQuery("delete from dbo.OnlineEducationSite_question")

# 添加其他数据

In [3]:
# knowledge 知识点
import json
ms = MSSQL(host=r'(local)',user=r'DRAKE',pwd=r'123456',db=r'question')
knowledgeSet = set()
with open('answer.txt', 'r') as lf:
    for line in lf:
        js = json.loads(line)
        value = js['value']
        for kl in value['knowledge']:
            knowledgeSet.add(kl)
count = 0
for knowledge in knowledgeSet:
    resList = ms.ExecQuery("select * from dbo.OnlineEducationSite_knowledge where knowledge='%s'" % knowledge)
    if len(resList) == 0:
        ms.ExecNonQuery("insert into dbo.OnlineEducationSite_knowledge values('%s','')" % knowledge)
    count += 1
    if count % 1000 == 0:
        print '已保存%d条数据！'% count
print '保存完成！一共保存了%d条数据！'% count

已保存1000条数据！
保存完成！一共保存了1185条数据！


In [5]:
# difficulty 难度
import json
ms = MSSQL(host=r'(local)',user=r'DRAKE',pwd=r'123456',db=r'question')
IdSet = set()
with open('answer.txt', 'r') as lf:
    for line in lf:
        js = json.loads(line)
        Id = js['value']['id']
        IdSet.add(Id)
count = 0
for Id in IdSet:
    resList = ms.ExecQuery("select * from dbo.OnlineEducationSite_difficulty where question_id='%s'" % Id)
    if len(resList) == 0:
        ms.ExecNonQuery("insert into dbo.OnlineEducationSite_difficulty values('%s',0,0)" % Id)
    count += 1
    if count % 1000 == 0:
        print '已保存%d条数据！'% count
print '保存完成！一共保存了%d条数据！'% count

已保存1000条数据！
已保存2000条数据！
已保存3000条数据！
已保存4000条数据！
保存完成！一共保存了4969条数据！


In [5]:
# description 描述
import json
ms = MSSQL(host=r'(local)',user=r'DRAKE',pwd=r'123456',db=r'question')
IdSet = set()
with open('answer.txt', 'r') as lf:
    for line in lf:
        js = json.loads(line)
        Id = js['value']['id']
        IdSet.add(Id)
count = 0
for Id in IdSet:
    resList = ms.ExecQuery("select * from dbo.OnlineEducationSite_description where question_id='%s'" % Id)
    if len(resList) == 0:
        ms.ExecNonQuery("insert into dbo.OnlineEducationSite_description values('','%s')" % Id)
    count += 1
    if count % 1000 == 0:
        print '已保存%d条数据！'% count
print '保存完成！一共保存了%d条数据！'% count
    

已保存1000条数据！
已保存2000条数据！
已保存3000条数据！
已保存4000条数据！
保存完成！一共保存了4969条数据！
